In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2

#### Data Loading

In [2]:
data_df = pd.read_csv("./labels.csv")
data_df.head()

,filepath,xmin,xmax,ymin,ymax
0,./images_labeled_xml\N1.xml,1093,1396,645,727
1,./images_labeled_xml\N100.xml,134,301,312,350
2,./images_labeled_xml\N101.xml,31,139,128,161
3,./images_labeled_xml\N102.xml,164,316,216,243
4,./images_labeled_xml\N103.xml,813,1067,665,724


In [3]:
import xml.etree.ElementTree as xet

In [4]:
filename = data_df['filepath'][0]
filename

'./images_labeled_xml\\N1.xml'

In [5]:
def get_filename(filename):
    filename_img = xet.parse(filename).getroot().find('filename').text
    filepath_img = os.path.join('./images_labeled_xml', filename_img)
    return filepath_img

In [6]:
print(get_filename(filename))

./images_labeled_xml\N1.jpeg


In [7]:
images_path = list(data_df['filepath'].apply(get_filename))
images_path[:5]

['./images_labeled_xml\\N1.jpeg',
 './images_labeled_xml\\N100.jpeg',
 './images_labeled_xml\\N101.jpeg',
 './images_labeled_xml\\N102.jpeg',
 './images_labeled_xml\\N103.jpeg']

#### Data Preprocessing

In [8]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [9]:
labels = data_df.iloc[:,1:].values
labels[:5]

array([[1093, 1396,  645,  727],
       [ 134,  301,  312,  350],
       [  31,  139,  128,  161],
       [ 164,  316,  216,  243],
       [ 813, 1067,  665,  724]], dtype=int64)

In [11]:
input = []
output = []

for i in range(len(images_path)):

    image = cv2.imread(images_path[i])
    h, w, d = image.shape

    # Image Preprocessing
    load_image = load_img(images_path[i], target_size=(224,224))
    image_array = img_to_array(load_image)
    norm_image_array = image_array/255.0
    input.append(norm_image_array)

    # Labels Preprocessing
    xmin, xmax, ymin, ymax = labels[i]
    nxmin, nxmax = xmin/w, xmax/w
    nymin, nymax = ymin/h, ymax/h

    label_norm = (nxmin, nxmax, nymin, nymax)
    output.append(label_norm)

In [13]:
X = np.array(input, dtype=np.float32)
y = np.array(output, dtype=np.float32)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=0)

#### Model Building

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.applications import MobileNetV2, InceptionV3, InceptionResNetV2
from keras.layers import Dense, Dropout, Flatten, Input
from keras.models import Model

In [ ]:
inception_resnet = InceptionResNetV2(include_top=False, weights="imagenet", 
                                     input_tensor=Input(shape=(224, 224, 3)))

inception_resnet.trainable = False

head_model = inception_resnet.output
head_model = Flatten()(head_model)
head_model = Dense(500, activation="relu")(head_model)
head_model = Dense(250, activation="relu")(head_model)
head_model = Dense(4, activation="sigmoid")(head_model)

model = Model(inputs=inception_resnet.input, outputs=head_model)

In [ ]:
model.compile(loss="mse", optimizer="Adam")
model.summary()

#### Model Training

In [ ]:
from tensorflow.keras.callbacks import TensorBoard

In [ ]:
tfb = TensorBoard("object-detection")

In [ ]:
history = model.fit(x=X_train, y=y_train, batch_size=10, 
                    epochs=10, validation_data=(X_test, y_test)
                    callbacks=[tfb])

#### Saving Model

In [ ]:
model.save('./models/object_detection.h5')